In [20]:
!head logs/qss* -n 5

==> logs/qss.log <==
[logfiles]
ParallelRoot : qss.log00 
top : qss.log03 
qssx : qss.log01 
qssy : qss.log02 

==> logs/qss.log00 <==
0 / L / D / 00:00:00:000:0 / top(03) / 00:00:00:000:0 / ParallelRoot(00)
0 / L / Y / 00:00:00:000:0 / top(03) / signaloutx / [0.5, 0] / ParallelRoot(00)
0 / L / Y / 00:00:00:000:0 / top(03) / signalouty / [0, 0] / ParallelRoot(00)
0 / L / D / 00:00:00:000:0 / top(03) / 00:00:00:000:0 / ParallelRoot(00)
0 / L / D / 00:00:00:000:0 / top(03) / 00:00:00:000:0.2 / ParallelRoot(00)

==> logs/qss.log01 <==
0 / L / I / 00:00:00:000:0 / top(03) / qssx(01)
0 / L / @ / 00:00:00:000:0 / top(03) / qssx(01)
0 / L / X / 00:00:00:000:0 / top(03) / in / [0, 0] / qssx(01)
0 / L / * / 00:00:00:000:0 / top(03) / qssx(01)
0 / L / X / 00:00:00:000:0.2 / top(03) / in / [-0.0001, 0] / qssx(01)

==> logs/qss.log02 <==
0 / L / I / 00:00:00:000:0 / top(03) / qssy(02)
0 / L / @ / 00:00:00:000:0 / top(03) / qssy(02)
0 / L / X / 00:00:00:000:0 / top(03) / in / [0.5, 0] / qssy(02)
0 

In [2]:
# Leemos el archivo de configuracion del log, tipicamente es el que no tiene indice.


In [3]:
config.sections()


['logfiles']

[('parallelroot', 'qss.log00'),
 ('top', 'qss.log03'),
 ('qssx', 'qss.log01'),
 ('qssy', 'qss.log02')]

Mapeamos el archivo de logs para parallelRoot, topLevel y listamos todos los otros modelos.

In [5]:

atomics

{'qssx': 'qss.log01', 'qssy': 'qss.log02'}

Vamos a procesar los mensajes Y, recordemos del libro de Wainer:
* messages signal the occurrence of internal events.
* X messages carry information about external input events.
* Y messages transmit the model’s output events.
* done messages carry scheduling information for future events, indicating that a model has finished with its current task.


In [47]:
import pandas as pd
def df_mensajes_log_a_df(patron, nombrearchivo):
    ! echo $nombrearchivo
    mensajes = !grep '$patron' $nombrearchivo
    qss_mensajes = pd.DataFrame([val.split(' / ') for val in mensajes])
    return qss_mensajes

def obtener_patrones(nombrearchivo):
    ! awk ' { print $1 " " $2 " " $3 " " $4 " " $5 } ' $nombrearchivo  | sort -u
    return patrones

def nombres_modelos(logfile_name):
    import os
    logdir_name = os.path.dirname(logfile_name)
    import ConfigParser, os
    config = ConfigParser.ConfigParser()
    config.readfp(open(logfile_name))
    
    fn_parallelroot = config.get('logfiles', 'parallelroot')
    fn_top = config.get('logfiles', 'top')

    atomics = { mod_name: os.path.join(logdir_name, fn_mod) for (mod_name, fn_mod) 
               in config.items('logfiles') 
               if mod_name not in ("top", "parallelroot")}
    return atomics

In [24]:
logs = []
log_dir = os.path.join('logs', 'qss1', 'teacup')
head_log = os.path.join(log_dir, 'log')
info_atomicos = nombres_modelos(head_log)

In [51]:
#{ modelo: {mensajes X: dataframe, mensajes Y: dataframe}}
modelos = {}
patrones = ['0 / L / X', 
            '0 / L / @',
            '0 / L / *',
            '0 / L / D',
            '0 / L / I',
            '0 / L / Y']

for nombre_atomico, archivo_log in info_atomicos.items():
    for patron in patrones:
        modelos[nombre_atomico] = modelos.get(nombre_atomico, {patron: None})
        modelos[nombre_atomico][patron] = df_mensajes_log_a_df(patron, archivo_log)

logs/qss1/teacup/log01
logs/qss1/teacup/log01
logs/qss1/teacup/log01
logs/qss1/teacup/log01
logs/qss1/teacup/log01
logs/qss1/teacup/log01
logs/qss1/teacup/log03
logs/qss1/teacup/log03
logs/qss1/teacup/log03
logs/qss1/teacup/log03
logs/qss1/teacup/log03
logs/qss1/teacup/log03
logs/qss1/teacup/log02
logs/qss1/teacup/log02
logs/qss1/teacup/log02
logs/qss1/teacup/log02
logs/qss1/teacup/log02
logs/qss1/teacup/log02
logs/qss1/teacup/log04
logs/qss1/teacup/log04
logs/qss1/teacup/log04
logs/qss1/teacup/log04
logs/qss1/teacup/log04
logs/qss1/teacup/log04


In [52]:
modelos

{'fm': {'0 / L / *':       0  1  2                         3        4       5          6       7
  0     0  L  I            00:00:00:000:0  top(05)  fm(04)       None    None
  1     0  L  X            00:00:00:000:0  top(05)  invar1       [80]  fm(04)
  2     0  L  *            00:00:00:000:0  top(05)  fm(04)       None    None
  3     0  L  @            00:00:00:000:0  top(05)  fm(04)       None    None
  4     0  L  *            00:00:00:000:0  top(05)  fm(04)       None    None
  5     0  L  X            00:00:00:800:0  top(05)  invar1     [79.2]  fm(04)
  6     0  L  *            00:00:00:800:0  top(05)  fm(04)       None    None
  7     0  L  @            00:00:00:800:0  top(05)  fm(04)       None    None
  8     0  L  *            00:00:00:800:0  top(05)  fm(04)       None    None
  9     0  L  X  00:00:08:720:6.10352e-05  top(05)  invar1   [78.408]  fm(04)
  10    0  L  *  00:00:08:720:6.10352e-05  top(05)  fm(04)       None    None
  11    0  L  @  00:00:08:720:6.10352e-05  to

In [39]:
modelos


{}

In [30]:
modelos

{'fm':      0  1  2                         3        4       5          6       7
 0    0  L  X            00:00:00:000:0  top(05)  invar1       [80]  fm(04)
 1    0  L  X            00:00:00:800:0  top(05)  invar1     [79.2]  fm(04)
 2    0  L  X  00:00:08:720:6.10352e-05  top(05)  invar1   [78.408]  fm(04)
 3    0  L  X     00:00:17:242:0.608826  top(05)  invar1  [77.6239]  fm(04)
 4    0  L  X     00:00:26:474:0.758911  top(05)  invar1  [76.8477]  fm(04)
 5    0  L  X     00:00:36:554:0.570961  top(05)  invar1  [76.0792]  fm(04)
 6    0  L  X     00:00:47:664:0.785194  top(05)  invar1  [75.3184]  fm(04)
 7    0  L  X     00:01:00:054:0.303932  top(05)  invar1  [74.5652]  fm(04)
 8    0  L  X      00:01:14:074:0.50943  top(05)  invar1  [73.8196]  fm(04)
 9    0  L  X     00:01:30:244:0.475174  top(05)  invar1  [73.0814]  fm(04)
 10   0  L  X     00:01:49:377:0.852249  top(05)  invar1  [72.3506]  fm(04)
 11   0  L  X      00:02:12:857:0.77742  top(05)  invar1  [71.6271]  fm(04)
 12   

In [4]:
import os
for log_file in os.listdir(log_dir):
    if str.startswith(log_file, 'log'):
        logs.append(log_file)
#df_mensajes_log_a_df('logs/qss1/teacup/')

['log03', 'log02', 'log', 'log05', 'log01', 'log04', 'log00']